# Stability testing with Tangent Plane Distance (TPD) function

The [tangent plane distance](https://www.sciencedirect.com/science/article/pii/0378381282850012) ($tpd$) function allows testing the relative stability of a phase of composition $z$ against a trial phase of composition $w$ at fixed temperature and pressure].


$$ tpd(\underline{w}) =  \sum_{i=1}^c w_i (\ln w_i +  \ln \hat{\phi}_i(\underline{w})
- \ln z_i - \ln \hat{\phi}_i(\underline{z})) $$

Usually, this function is minimized to check the stability of the given phase based on the following criteria:
- If the minimized $tpd$ is positive, the global phase $z$ is stable.
- If the minimized $tpd$ is zero, the global phase $z$ and trial phase $w$ are in equilibrium.
- If the minimized $tpd$ is negative, the global phase $z$ is unstable


In this notebook, stability analysis for the mixture of water and butanol will be performed. To start, the required functions are imported.

In [1]:
import numpy as np
from SGTPy import component, mixture, saftvrmie
from SGTPy.equilibrium import tpd_min, tpd_minimas, lle_init

Then, the mixture of water and butanol and its interaction parameters are set up.

In [2]:
# creating pure components
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)
mix = mixture(water, butanol)

# optimized from experimental LLE
kij, lij = np.array([-0.00736075, -0.00737153])
Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])

# setting interactions corrections
mix.kij_saft(Kij)
mix.lij_saft(Lij)
# creating eos model
eos = saftvrmie(mix)

----
### tpd_min

The ``tpd_min`` function searches for a phase composition corresponding to a minimum of $tpd$ function given an initial value. The user needs to specify whether the trial (W) and reference (Z) phases are liquids (``L``) or vapors (``V``).

In [3]:
T = 320 # K
P = 1.01e5 # Pa
z = np.array([0.8, 0.2])
#Search for trial phase
w = np.array([0.99, 0.01])
tpd_min(w, z, T, P, eos, stateW = 'L', stateZ = 'L')
#composition of minimum found and tpd value
#(array([0.95593129, 0.04406871]), -0.011057873031562693)

(array([0.95593128, 0.04406872]), -0.011057869928527753)

In [4]:
w = np.array([0.99, 0.01])
tpd_min(w, z, T, P, eos, stateW = 'V', stateZ = 'L')
#composition of minimum found and tpd value
#(array([0.82414873, 0.17585127]), 0.8662934867235452)

(array([0.82414986, 0.17585014]), 0.8662935076215186)

---
### tpd_minimas
The ``tpd_minimas`` function will attempt (but does not guarantee) to search for ``nmin`` minima of the $tpd$ function. As for the ``tpd_min`` function, you need to specify the aggregation state of the global (``z``) and the trial phase (``w``).

In [5]:
T = 320 # K
P = 1.01e5 # Pa
z = np.array([0.8, 0.2])
nmin = 2
tpd_minimas(nmin, z, T, P, eos, stateW='L', stateZ='L')

((array([0.95593125, 0.04406875]), array([0.55571877, 0.44428123])),
 array([-0.01105787, -0.01083625]))

In [6]:
tpd_minimas(nmin, z, T, P, eos, stateW='V', stateZ='L')

((array([0.82414943, 0.17585057]), array([0.82414943, 0.17585057])),
 array([0.86629351, 0.86629351]))

---
### lle_init

Finally, the ``lle_init`` function can be used to find initial guesses for liquid-liquid equilibrium calculation.

This function call ``tpd_minimas`` with ``nmin=2`` and liquid state for trial and global phase.

In [7]:
T = 320 # K
P = 1.01e5 # Pa
z = np.array([0.8, 0.2])
lle_init(z, T, P, eos)

(array([0.95593125, 0.04406875]), array([0.55572869, 0.44427131]))

---
For further information about each function check out the documentation running: ``function?``